<a href="https://colab.research.google.com/github/bbang3/korean-text-augmentation/blob/klue/experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/korean-text-augmentation

/content/drive/MyDrive/korean-text-augmentation


# Import

In [4]:
!pip install transformers[pytorch]
!pip install datasets
!pip install accelerate
!pip install evaluate
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.8 MB/s eta 0:00:00


In [30]:
import os
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm

from transformers import BertModel, BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate

import wandb

In [6]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
os.environ["WANDB_PROJECT"] = "KLUE - KRBERT"

In [72]:
train_path = 'data/train_mlm_2.csv'
val_path = 'data/val_low.csv'
test_path = 'data/test.csv'
dataset = load_dataset('csv', data_files={'train': train_path, 'validation': val_path, 'test': test_path})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [73]:
import gc
gc.collect()
torch.cuda.empty_cache()
del model

In [74]:
num_labels = 7
tokenizer = BertTokenizer.from_pretrained("snunlp/KR-Medium")
model = BertForSequenceClassification.from_pretrained("snunlp/KR-Medium", num_labels=num_labels).to("cuda")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-Medium and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [75]:
def tokenization(example):
    return tokenizer(example["title"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenization, batched=True, remove_columns=["title"])
tokenized_dataset.set_format("torch")

Map:   0%|          | 0/840 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [76]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [77]:
training_args = TrainingArguments(
    output_dir='./checkpoints/mlm_no_train',
    num_train_epochs=4,
    warmup_steps=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    seed=42,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    report_to='wandb',
    run_name='mlm_no_train',
    save_total_limit=2,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    compute_metrics=compute_metrics,
)

In [78]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.758800,1.173609,0.700000
2,0.589300,0.851395,0.728571
3,0.118000,1.255342,0.742857
4,0.024900,1.326895,0.714286


TrainOutput(global_step=212, training_loss=0.6227620851318791, metrics={'train_runtime': 36.5566, 'train_samples_per_second': 91.912, 'train_steps_per_second': 5.799, 'total_flos': 35320133626800.0, 'train_loss': 0.6227620851318791, 'epoch': 4.0})

In [79]:
wandb.finish()

eval/accuracy,▁▆█▃
eval/loss,▆▁▇█
eval/runtime,▃█▁█
eval/samples_per_second,▆▁█▁
eval/steps_per_second,▆▁█▁
train/epoch,▁▁▃▃▆▆███
train/global_step,▁▁▃▃▆▆███
train/learning_rate,▅█▄▁
train/loss,█▃▁▁
train/total_flos,▁
train/train_loss,▁


# Load model

In [80]:
trainer.state.best_model_checkpoint

'./checkpoints/mlm_no_train/checkpoint-159'

In [81]:
num_labels = 7
tokenizer = BertTokenizer.from_pretrained("snunlp/KR-Medium")
ft_model = BertForSequenceClassification.from_pretrained(trainer.state.best_model_checkpoint, num_labels=num_labels).to("cuda")
# ft_model = BertForSequenceClassification.from_pretrained("checkpoints/bt_2/checkpoint-848", num_labels=num_labels).to("cuda")

In [82]:
training_args = TrainingArguments(
    output_dir='./checkpoints/tmp',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    seed=42,
    evaluation_strategy='no',
    logging_strategy='epoch',
    save_strategy='no',
    report_to='wandb',
    run_name='test-mlm-no-train'
)

trainer = Trainer(
    model=ft_model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics
)

In [83]:
trainer.evaluate(eval_dataset=tokenized_dataset['test'])

{'eval_loss': 0.9337495565414429,
 'eval_accuracy': 0.7824285714285715,
 'eval_runtime': 9.406,
 'eval_samples_per_second': 744.205,
 'eval_steps_per_second': 46.566}

In [71]:
wandb.finish()

eval/accuracy,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/global_step,▁
eval/accuracy,0.70729
eval/loss,1.16869
eval/runtime,9.8842
eval/samples_per_second,708.197
eval/steps_per_second,44.313


In [51]:
predictions = trainer.predict(test_dataset=tokenized_dataset['validation'])
predictions = np.argmax(predictions.predictions, axis=-1)
predictions

array([5, 3, 1, 5, 2, 6, 0, 3, 0, 6, 3, 1, 3, 0, 3, 3, 5, 3, 0, 3, 5, 2,
       4, 2, 5, 0, 5, 4, 1, 1, 2, 1, 1, 2, 6, 6, 3, 5, 1, 5, 5, 3, 3, 3,
       1, 3, 3, 3, 3, 3, 6, 3, 3, 4, 3, 4, 4, 3, 3, 1, 3, 0, 3, 6, 6, 3,
       4, 4, 3, 3])

In [53]:
gt = tokenized_dataset['validation']['label']
gt

tensor([5, 2, 2, 4, 0, 6, 2, 0, 0, 4, 2, 0, 4, 2, 3, 6, 5, 2, 0, 1, 5, 6, 5, 4,
        2, 3, 5, 4, 1, 1, 0, 1, 1, 5, 6, 3, 3, 3, 1, 4, 1, 2, 3, 4, 0, 3, 3, 2,
        5, 5, 6, 0, 6, 4, 0, 5, 1, 2, 4, 1, 4, 0, 6, 6, 6, 3, 5, 6, 1, 3])

In [20]:
df_eda = pd.read_csv('data/train_eda_2.csv')
df_eda

,guid,title,label
0,ynat-v1_train_00401,홍남기 부총리 출입기자단과 간담회,6
1,ynat-v1_train_00401,홍남기 부총리 출입기자단과 간담회,6
2,ynat-v1_train_00401,홍남기 부총리 출입기자단과 간담집단,6
3,ynat-v1_train_00492,대구 예비후보 의정보고회 금지 가처분 신청,6
4,ynat-v1_train_00492,대구 예비후보 의정보고학회 금지 가처분 신청,6
...,...,...,...
835,ynat-v1_train_45312,예루살렘 제거 팔레스타인 주거지 철거 이스라엘에 비난 쇄도,4
836,ynat-v1_train_45312,예루살렘 팔레스타인 주거지 절제 이스라엘에 비난 쇄도,4
837,ynat-v1_train_45546,사랑의 김장 들어보이는 이해찬,6
838,ynat-v1_train_45546,성애의 김장 들어보이는 이해찬,6


In [22]:
# Count duplicates
df_eda.duplicated()

0      False
1       True
2      False
3      False
4      False
       ...  
835    False
836    False
837    False
838    False
839    False
Length: 840, dtype: bool